In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import CustomObjectScope
import segmentation_models as sm
from keras.metrics import MeanIoU
import glob
import cv2
import os
from matplotlib import pyplot as plt

In [ ]:
BACKBONE1 = 'resnet101'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

SIZE_X = 256
SIZE_Y = 256
n_classes = 2

In [ ]:
# Capture training image info as a list
test_images = []

for directory_path in glob.glob("test_images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 1)
        img = cv2.resize(img, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
        test_images.append(img)
    
#Convert list to array for machine learning processing        
test_images = np.array(test_images)

In [ ]:
# Capture mask/label info as a list
test_masks = [] 

for directory_path in glob.glob("test_masks"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        mask = cv2.imread(img_path, 1)
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
        mask[mask <= 15] = 0
        mask[mask >= 200] = 255
        test_masks.append(mask)
        
# Convert list to array for machine learning processing          
test_masks = np.array(test_masks)

In [ ]:
def Exp_loss(y_true, y_pred):
  y_pred = tf.nn.softmax(y_pred)

  true = 0.8 * tf.math.exp(-1 * tf.math.multiply(y_true, y_pred))
  wrong = tf.math.exp(-1 * tf.math.multiply((1 - y_true), (1 - y_pred)))
  loss = tf.reduce_mean(true + wrong)
  
  return loss

In [ ]:
def dice_coef(y_true, y_pred, smooth=1.0):
    # Flatten the input tensors
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    
    # Calculate intersection and union of the flattened tensors
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f)
    
    # Calculate Dice coefficient
    dice_coef = (2.0 * intersection + smooth) / (union + smooth)
    
    return dice_coef

In [ ]:
# Encode labels
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
n, h, w = test_masks.shape
test_masks_reshaped = test_masks.reshape(-1, 1)
test_masks_reshaped_encoded = labelencoder.fit_transform(test_masks_reshaped)
test_encoded_original_shape = test_masks_reshaped_encoded.reshape(n, h, w)

np.unique(test_encoded_original_shape)

In [ ]:
test_masks_input = np.expand_dims(test_encoded_original_shape, axis = 3)

In [ ]:
y_test = test_masks_input
X_test1 = preprocess_input1(test_images)

In [ ]:
with CustomObjectScope({'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5),  'Exp_loss': Exp_loss, 'dice_coef': dice_coef}):
    model1 = tf.keras.models.load_model("Main_Model.hdf5", compile=False)

In [ ]:
with CustomObjectScope({'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5), 'Exp_loss': Exp_loss, 'dice_coef': dice_coef}):
    model2 = tf.keras.models.load_model("Sampling_Model.hdf5", compile=False)

In [ ]:
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), dice_coef]

model1.compile(optimizer='adam', loss=Exp_loss, metrics=metrics)
model2.compile(optimizer='adam', loss=Exp_loss, metrics=metrics)

In [ ]:
class w_ave(tf.keras.layers.Layer):
    def __init__(self, w1, w2, **kwargs):
        super().__init__( **kwargs)
        self.w1 = w1
        self.w2 = w2
    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1]

In [ ]:
models = [model1, model2]
input = tf.keras.layers.Input(shape=(256, 256, 3), dtype='float32')
ensemble = [x(input) for x in models]
ensemble_output = w_ave(1.01, 1)(ensemble)
final_model = tf.keras.models.Model(inputs=input, outputs = ensemble_output)

In [ ]:
y_pred = final_model.predict(test_images)
y_pred_argmax = np.argmax(y_pred, axis = 3)

In [ ]:
n_classes = 2
IOU_keras = MeanIoU(num_classes = n_classes)
IOU_keras.update_state(y_test[:, :, :, 0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
# To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0] / (values[0,0] + values[0,1] + values[1,0])
class2_IoU = values[1,1] / (values[1,1] + values[1,0]  + values[0,1])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)

In [ ]:
#Test some random images

import random
test_img_number = random.randint(0, len(X_test1) - 1)
test_img = X_test1[test_img_number]
ground_truth=y_test[test_img_number]
test_img_input=np.expand_dims(test_img, 0)

test_img_input1 = preprocess_input1(test_img_input)

test_pred1 = final_model.predict(test_img_input1)
test_prediction1 = np.argmax(test_pred1, axis=3)[0,:,:]

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction1, cmap='jet')
plt.show()